<a href="https://colab.research.google.com/github/maria-natale/ProgettoFVAB/blob/main/Code/3blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/maria-natale/ProgettoFVAB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.preprocessing import image
from PIL import Image

path_git = '/content/ProgettoFVAB'
filename = 'spagnolo_giapponese'
path_drive = '/content/drive/MyDrive/Casillo&Natale'
dataset_dir = 'dataset_Spagnolo_giapponese'

In [3]:
train = pd.read_csv(os.path.join(path_git, 'file_dataset', filename+'ImmaginiClasse.csv'))
path_images = os.path.join(path_drive, dataset_dir, 'train')
train.head()

,frame,lingua
0,4_2_1_25_30_1_m.avi_frame143.jpg,Spagnolo
1,4_2_1_25_30_1_m.avi_frame144.jpg,Spagnolo
2,4_2_1_25_30_1_m.avi_frame145.jpg,Spagnolo
3,4_2_1_25_30_1_m.avi_frame146.jpg,Spagnolo
4,4_2_1_25_30_1_m.avi_frame147.jpg,Spagnolo


In [ ]:
from PIL import Image
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    try:
      img = image.load_img(path_images+'/'+train['frame'][i], target_size=(224,224,3))
      # converting it to array
      img = image.img_to_array(img)
      # normalizing the pixel value
      img = img/255
      # appending the image to the train_image list
      train_image.append(img)
    except :
      print("corrotta")
    #print(img)
    
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

  1%|          | 229/41152 [00:00<02:43, 250.03it/s]

corrotta


 20%|█▉        | 8151/41152 [25:31<2:00:32,  4.56it/s]

In [2]:
pip install Pillow==8.2.0

In [11]:
print(train['frame'][243])
print(train['frame'][244])

4_2_1_25_30_1_m.avi_frame413.jpg
4_2_1_25_30_2_m.avi_frame440.jpg
